<a href="https://colab.research.google.com/github/Gilf641/EVA4/blob/master/S4_AssignmentSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the required libraries

from __future__ import print_function
import torch # PyTorch Main Library
import torch.nn as nn # Neural Net Lib
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms 

Here I have used 3 convolution Blocks

In [37]:
# Main Model 

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # Conv Block 1 
      self.conv1 = nn.Conv2d(1, 16, 3) # 26
      self.bn1 = nn.BatchNorm2d(16)
      self.conv2 = nn.Conv2d(16, 16, 3) # 24
      self.bn2 = nn.BatchNorm2d(16)
      self.pool1 = nn.AvgPool2d(2,2) # 12 # AvgPool2d Instead of MaxPool2d
      self.drop1 = nn.Dropout(0.10)


      # Conv Block 2
      self.conv3 = nn.Conv2d(16, 32, 1) # 12
      self.bn3 = nn.BatchNorm2d(32)
      self.conv4 = nn.Conv2d(32, 32, 3) # 10
      self.bn4 = nn.BatchNorm2d(32)
      self.drop2 = nn.Dropout(0.10)


     # Conv Block 3
      self.conv5 = nn.Conv2d(32, 16, 3) # 8
      self.bn5 = nn.BatchNorm2d(16)
      self.conv6 = nn.Conv2d(16, 16, 1) # 8
      self.bn6 = nn.BatchNorm2d(16)
      self.drop3 = nn.Dropout(0.10)


      self.conv7 = nn.Conv2d(16, 10, 1) # 8

      # adding Global Average Pooling
      self.gap1 = nn.AdaptiveAvgPool2d(1)


    def forward(self, x):
      x = self.drop1(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
      x = self.drop2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
      x = self.drop3(self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x)))))))

      x = self.gap1(self.conv7(x))
      x = x.view(-1, 10)
      
      return F.log_softmax(x)


!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model2 = Net().to(device)
summary(model2, input_size=(1, 28, 28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
         AvgPool2d-5           [-1, 16, 12, 12]               0
           Dropout-6           [-1, 16, 12, 12]               0
            Conv2d-7           [-1, 32, 12, 12]             544
       BatchNorm2d-8           [-1, 32, 12, 12]              64
            Conv2d-9           [-1, 32, 10, 10]           9,248
      BatchNorm2d-10           [-1, 32, 10, 10]              64
          Dropout-11           [-1, 32, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           4,624
      BatchNorm2d-13             [-1, 16, 8, 8]              32
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
# Data Loading and Pre-Processing

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(

    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
# Pre-Train & Test Code

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Epoch {} \nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [40]:
# Optimization and Testing
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 1 
Test set: Average loss: 0.0903, Accuracy: 9808/10000 (98.08%)



  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 2 
Test set: Average loss: 0.0568, Accuracy: 9851/10000 (98.51%)



  0%|          | 0/469 [00:00<?, ?it/s]

...

Epoch 18 
Test set: Average loss: 0.0237, Accuracy: 9933/10000 (99.33%)



  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 19 
Test set: Average loss: 0.0186, Accuracy: 9942/10000 (99.42%)



loss=0.027452493086457253 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.52it/s]


Epoch 20 
Test set: Average loss: 0.0184, Accuracy: 9939/10000 (99.39%)

